In [1]:
import itertools
import numpy as np
from itertools import permutations 
import json

from classes import *
from frank_main import *
EVA = True

24-Feb-21 00:55:52 fatf.utils.array.tools INFO     Using numpy's numpy.lib.recfunctions.structured_to_unstructured as fatf.utils.array.tools.structured_to_unstructured and fatf.utils.array.tools.structured_to_unstructured_row.


In [2]:
RULE = True
PAST = True
SKEPT = True
GROUP = True

N_BINS = 10
N_VAR = 3
MAX = 5

#predict_mode = 100
#believe_mode = 0


In [3]:
dataset = 'HR'

In [5]:
KNN = False

if dataset == 'adult':
    target = 'Probability'
    protected = ['sex']
    rule_att = 'capital-gain'
    if KNN:
        rule_value = 9000
    else:
        rule_value = 0.086141
    cats = ['workclass', 'marital-status', 'occupation', 'relationship?', 'race', 'sex',
       'native-country']
elif dataset == 'HR':
    target = 'is_promoted'
    protected = ['gender']
    rule_att = 'awards_won?'
    if KNN:
        rule_value = 0
    else:
        rule_value = 0
    cats = ['awards_won?', 'gender', 'department', 'recruitment_channel']
elif dataset == 'compas':
    target = 'two_year_recid'
    protected = ['sex']
    rule_att = 'priors_count'
    if KNN:
        rule_value = 0
    else:
        rule_value = 0
    cats = ['sex', 'race', 'c_charge_degree']
    

df = pd.read_json("bot_"+dataset+"_stream.json")
user_train = pd.read_json("bot_"+dataset+"_user_train.json")
frank_train = pd.read_json("bot_"+dataset+"_frank_train.json")
test = pd.read_json("bot_"+dataset+"_test.json")

model = GaussianNB()

#training the user
Y_user_train = list(user_train[target])
Y_user_train = [bool(y) for y in Y_user_train]
X_user_train = user_train.loc[:, user_train.columns!=target]
X_user_train = X_user_train.values.tolist()

model.fit(X_user_train, Y_user_train) 

#training Frank
Y_frank_train = list(frank_train[target])
Y_frank_train = [bool(y) for y in Y_frank_train]
X_frank_train = frank_train.loc[:, frank_train.columns!=target]
X_frank_train = list(X_frank_train.to_dict(orient='index').values())
    
#test set (for Frank)
Y_test = list(test[target])
Y_test = [bool(y) for y in Y_test]
X_test = test.loc[:, test.columns!=target]
X_test = list(X_test.to_dict(orient='index').values())




In [6]:
believe_value = 0.8 #For Frank Suggestions
rethink_value = 0.8 #Individual Fairness
fairness_value = 0.25 #top records to re-label (Group Fairness)

expertise_value = 0.5

In [7]:
#user_model = GroundTruther(believe_value, rethink_value, fairness_value, expertise_value)
user_model = ModelBased(believe_value, rethink_value, fairness_value, GaussianNB(), X_user_train, Y_user_train)

In [8]:
f = Frank(user_model, df, target, protected, cats, rule_att, rule_value, RULE, PAST, SKEPT, GROUP, X_test, Y_test, EVA, N_BINS, N_VAR, MAX)
f.train(X_frank_train, Y_frank_train)
results, eva = f.start()

Model trained


100%|███████████████████████████████████████| 2000/2000 [02:44<00:00, 12.13it/s]


In [9]:
results

{(1, 1, 0, 2, 2, 36, 4, 11, 0, 61): {'notes': [],
  'vs': None,
  'ideal': None,
  'times': 1,
  'dict_form': {'department': 1,
   'education': 1,
   'gender': 0,
   'recruitment_channel': 2,
   'no_of_trainings': 2,
   'age': 36,
   'previous_year_rating': 4,
   'length_of_service': 11,
   'awards_won?': 0,
   'avg_training_score': 61},
  'decision': False,
  'user': False,
  'machine': False},
 (4, 1, 1, 0, 2, 42, 3, 6, 0, 61): {'notes': [],
  'vs': None,
  'ideal': None,
  'times': 1,
  'dict_form': {'department': 4,
   'education': 1,
   'gender': 1,
   'recruitment_channel': 0,
   'no_of_trainings': 2,
   'age': 42,
   'previous_year_rating': 3,
   'length_of_service': 6,
   'awards_won?': 0,
   'avg_training_score': 61},
  'decision': False,
  'user': False,
  'machine': False},
 (4, 2, 0, 2, 1, 37, 4, 7, 0, 56): {'notes': [],
  'vs': None,
  'ideal': None,
  'times': 1,
  'dict_form': {'department': 4,
   'education': 2,
   'gender': 0,
   'recruitment_channel': 2,
   'no_of_tra

In [10]:
eva

[[0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 2, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 3, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 4, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 5, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 6, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 7, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 8, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 9, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 10, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 11, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 12, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 13, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 14, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 15, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 16, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 17, 0],
 [0.0, 1.0, 0, 0.0, 0.93, 0, 0, 0, 0, 0, 0, 0, 18, 0],
 [0.0, 1.0, 0, 0.0, 0